In [142]:
import torch
import numpy as np
from tqdm import tqdm

In [ ]:

# Define alphas as torch tensors with requires_grad=True
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True)
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)

alphas = [alpha_1, alpha_2, alpha_3, alpha_4]

# psi function using torch
def psi(x, y, alpha_1, alpha_2, alpha_3, alpha_4):
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

# Tensors with gradients enabled
x = torch.tensor([0.2, 0.3, 0.1], dtype=torch.float64, requires_grad=True)
y = torch.tensor([0.1, 0.4, 0.3], dtype=torch.float64, requires_grad=True)

# Compute psi
psi_val = psi(x, y, *alphas)

# First gradients w.r.t. x and y
grad_x = torch.autograd.grad(psi_val, x, create_graph=True)[0]
grad_y = torch.autograd.grad(psi_val, y, create_graph=True)[0]

# Laplacians
laplacian_x = sum(torch.autograd.grad(grad_x[i], x, retain_graph=True, create_graph=True)[0][i] for i in range(3))
laplacian_y = sum(torch.autograd.grad(grad_y[i], y, retain_graph=True, create_graph=True)[0][i] for i in range(3))

lap = laplacian_x + laplacian_y
# Derivatives of Laplacians w.r.t. alphas
dlap_dalpha = torch.autograd.grad(lap, alphas, retain_graph=True)


print(dlap_dalpha)

(tensor(-0.5851, dtype=torch.float64), tensor(2.5131, dtype=torch.float64), tensor(0.6771, dtype=torch.float64), tensor(-2.5998, dtype=torch.float64))


In [165]:
"""
def local_energy(psi_val, r1, r2):
    grad_x = torch.autograd.grad(psi_val, r1, create_graph=True)[0]
    grad_y = torch.autograd.grad(psi_val, r2, create_graph=True)[0]

    # Laplacians
    laplacian_x = sum(torch.autograd.grad(grad_x[i], r1, retain_graph=True, create_graph=True)[0][i] for i in range(3))
    laplacian_y = sum(torch.autograd.grad(grad_y[i], r2, retain_graph=True, create_graph=True)[0][i] for i in range(3))

    ke = - 0.5 * (laplacian_x + laplacian_y)

    return - 2 / torch.norm(r1) - 2 / torch.norm(r2) + 1 / torch.norm(r1 - r2) + ke
"""

def local_energy(psi_val, r1, r2):
    grad_r1 = torch.autograd.grad(psi_val, r1, create_graph=True)[0]
    grad_r2 = torch.autograd.grad(psi_val, r2, create_graph=True)[0]

    lap_r1 = torch.autograd.grad(grad_r1, r1, grad_outputs=torch.ones_like(grad_r1), create_graph=True)[0]
    lap_r2 = torch.autograd.grad(grad_r2, r2, grad_outputs=torch.ones_like(grad_r2), create_graph=True)[0]

    laplacian = lap_r1.sum() + lap_r2.sum()
    ke = -0.5 * laplacian

    potential = -2 / torch.norm(r1) - 2 / torch.norm(r2) + 1 / torch.norm(r1 - r2)

    return ke + potential


In [171]:
L = 1
r1 = torch.rand(3, requires_grad=True) * 2 * L - L
r2 = torch.rand(3, requires_grad=True) * 2 * L - L #random number from -L to L
E = 0
E2 = 0
Eln_average = 0
ln_average = 0
rejection_ratio = 0
step = 0
max_steps = 500
N = 10000
dlap_dalpha = 0
xs = []
ys = []
psi_vals = []

for i in tqdm(range(N)):
    flag = False
    chose = np.random.rand()
    step = step + 1
    if chose < 0.5:
        r1_trial = r1 + 0.5 * (torch.rand(3) * 2 * L-L)
        r2_trial = r2
    else:
        r2_trial = r2 + 0.5 * (torch.rand(3) * 2 * L-L)
        r1_trial = r1


    psi_val = psi(r1, r2, *alphas)
    psi_val_trial = psi(r1_trial, r2_trial, *alphas)
    

    if psi_val_trial.item() >= psi_val.item():
        r1 = r1_trial
        r2 = r2_trial
        flag = True
    
    else:
        dummy = np.random.rand()
        if dummy < psi_val_trial.item() / psi_val.item():
            r1 = r1_trial
            r2 = r2_trial
        else:
            rejection_ratio += 1./N
            
    if step > max_steps:
        xs.append(r1)
        ys.append(r2)

        if flag:
            psi_vals.append(psi_val_trial)
        else:
            psi_vals.append(psi_val)
        """
        local_E = local_energy(psi_val_trial, r1, r2)
        E += local_E / (N - max_steps)
        E2 += local_E ** 2 / (N - max_steps)
        dlap_dalpha += torch.tensor(torch.autograd.grad(local_E, alphas, retain_graph=True)) / (N - max_steps)
        """





  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:04<00:00, 2299.97it/s]


In [189]:
def batched_psi(x, y, alpha_1, alpha_2, alpha_3, alpha_4):
    r1 = torch.norm(x, dim=1)
    r2 = torch.norm(y, dim=1)
    r12 = torch.norm(x - y, dim=1)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

In [191]:
def batched_local_energy(r1_batch, r2_batch, *alphas):
    r1_batch = r1_batch.detach().requires_grad_()
    r2_batch = r2_batch.detach().requires_grad_()
    psi_batch = psi(r1_batch, r2_batch, *alphas)

    grad_r1 = torch.autograd.grad(psi_batch, r1_batch, grad_outputs=torch.ones_like(psi_batch), create_graph=True)[0]
    grad_r2 = torch.autograd.grad(psi_batch, r2_batch, grad_outputs=torch.ones_like(psi_batch), create_graph=True)[0]

    lap_r1 = torch.zeros_like(psi_batch)
    lap_r2 = torch.zeros_like(psi_batch)

    for i in range(3):
        lap_r1 += torch.autograd.grad(grad_r1[:, i], r1_batch, grad_outputs=torch.ones_like(psi_batch), retain_graph=True, create_graph=True)[0][:, i]
        lap_r2 += torch.autograd.grad(grad_r2[:, i], r2_batch, grad_outputs=torch.ones_like(psi_batch), retain_graph=True, create_graph=True)[0][:, i]

    kinetic = -0.5 * (lap_r1 + lap_r2)
    potential = (
        -2 / torch.norm(r1_batch, dim=1)
        -2 / torch.norm(r2_batch, dim=1)
        +1 / torch.norm(r1_batch - r2_batch, dim=1)
    )

    return kinetic + potential


In [193]:
local_E = batched_local_energy(torch.stack(xs[:5]), torch.stack(ys[:5]), *alphas)

RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([]) and output[0] has a shape of torch.Size([5]).

In [158]:
E

tensor(-2.7991, grad_fn=<AddBackward0>)

In [159]:
E2

tensor(10.8893, grad_fn=<AddBackward0>)

In [160]:
dlap_dalpha

tensor([-0.0043,  0.0316,  0.0081, -0.0188], dtype=torch.float64)